# Load Libraries

In [187]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [188]:
import random
random.seed(5)

# Load Data

In [223]:
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
X = iris.data
Y = iris.target

In [224]:
from sklearn.model_selection import train_test_split
x, x_val, y, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)

In [225]:
x.shape, y.shape, x_val.shape, y_val.shape

((100, 4), (100,), (50, 4), (50,))

In [226]:
x.shape, y.shape

((100, 4), (100,))

In [227]:
x_train = x.reshape(-1, x.shape[1]).astype('float32')
y_train = y

x_val = x_val.reshape(-1, x_val.shape[1]).astype('float32')
y_val = y_val

In [228]:
x_train.shape

(100, 4)

Define validation data as a Pytorch Tensor

In [229]:
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)

## Data Preparation für DL

Put Data through DataLoader, so we can use batches

In [230]:
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(x).type(torch.FloatTensor)
        self.y=torch.from_numpy(y).type(torch.LongTensor)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [231]:
data_set=Data()

In [232]:
trainloader=DataLoader(dataset=data_set,batch_size=10)

In [233]:
data_set.x[0:10]

tensor([[5.7000, 2.9000, 4.2000, 1.3000],
        [7.6000, 3.0000, 6.6000, 2.1000],
        [5.6000, 3.0000, 4.5000, 1.5000],
        [5.1000, 3.5000, 1.4000, 0.2000],
        [7.7000, 2.8000, 6.7000, 2.0000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [5.2000, 3.4000, 1.4000, 0.2000],
        [5.0000, 3.5000, 1.3000, 0.3000],
        [5.1000, 3.8000, 1.9000, 0.4000],
        [5.0000, 2.0000, 3.5000, 1.0000]])

In [234]:
data_set.y[0:10]

tensor([1, 2, 1, 0, 2, 1, 0, 0, 0, 1])

In [235]:
data_set.x.shape, data_set.y.shape

(torch.Size([100, 4]), torch.Size([100]))

# Build Model and train it

He Initialisation

In [236]:
class Net(nn.Module):
    def __init__(self,in_size,n_hidden1,n_hidden2,out_size,p=0):

        super(Net,self).__init__()
        self.drop=nn.Dropout(p=p)
        self.linear1=nn.Linear(in_size,n_hidden1)
        nn.init.kaiming_uniform_(self.linear1.weight,nonlinearity='relu')
        self.linear2=nn.Linear(n_hidden1,n_hidden2)
        nn.init.kaiming_uniform_(self.linear1.weight,nonlinearity='relu')
        self.linear3=nn.Linear(n_hidden2,n_hidden2)
        nn.init.kaiming_uniform_(self.linear3.weight,nonlinearity='relu')
        self.linear4=nn.Linear(n_hidden2,out_size)
        
    def forward(self,x):
        x=F.relu(self.linear1(x))
        x=self.drop(x)
        x=F.relu(self.linear2(x))
        x=self.drop(x)
        x=F.relu(self.linear3(x))
        x=self.drop(x)
        x=self.linear4(x)
        return x

In [237]:
model=Net(4,10,5,3)
model_drop=Net(4,10,5,3,p=0.2)
model_drop

Net(
  (drop): Dropout(p=0.2)
  (linear1): Linear(in_features=4, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=5, bias=True)
  (linear3): Linear(in_features=5, out_features=5, bias=True)
  (linear4): Linear(in_features=5, out_features=3, bias=True)
)

In [238]:
model_drop.train()

Net(
  (drop): Dropout(p=0.2)
  (linear1): Linear(in_features=4, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=5, bias=True)
  (linear3): Linear(in_features=5, out_features=5, bias=True)
  (linear4): Linear(in_features=5, out_features=3, bias=True)
)

In [239]:
optimizer_ofit = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer_drop = torch.optim.Adam(model_drop.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [240]:
LOSS={}
LOSS['training data no dropout']=[]
LOSS['validation data no dropout']=[]
LOSS['training data dropout']=[]
LOSS['validation data dropout']=[]

In [241]:
n_epochs=20

for epoch in range(n_epochs):
    for x, y in trainloader:
        #make a prediction for both models 
        yhat = model(data_set.x)
        yhat_drop = model_drop(data_set.x)
        #calculate the lossf or both models 
        loss = criterion(yhat, data_set.y)
        loss_drop = criterion(yhat_drop, data_set.y)

        #store the loss for  both the training and validation  data for both models 
        LOSS['training data no dropout'].append(loss.item())
        LOSS['training data dropout'].append(loss_drop.item())
        model_drop.eval()
        model_drop.train()

        #clear gradient 
        optimizer_ofit.zero_grad()
        optimizer_drop.zero_grad()
        #Backward pass: compute gradient of the loss with respect to all the learnable parameters
        loss.backward()
        loss_drop.backward()
        #the step function on an Optimizer makes an update to its parameters
        optimizer_ofit.step()
        optimizer_drop.step()
        
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 1.2261919975280762
epoch 0, loss 1.1832058429718018
epoch 0, loss 1.1479732990264893
epoch 0, loss 1.1195118427276611
epoch 0, loss 1.0988399982452393
epoch 0, loss 1.0859699249267578
epoch 0, loss 1.0753216743469238
epoch 0, loss 1.0665377378463745
epoch 0, loss 1.0593410730361938
epoch 0, loss 1.053403615951538
epoch 1, loss 1.0483787059783936
epoch 1, loss 1.0439419746398926
epoch 1, loss 1.0398415327072144
epoch 1, loss 1.0357444286346436
epoch 1, loss 1.031473159790039
epoch 1, loss 1.0264718532562256
epoch 1, loss 1.020595908164978
epoch 1, loss 1.013972282409668
epoch 1, loss 1.006752371788025
epoch 1, loss 0.9985687732696533
epoch 2, loss 0.9888967275619507
epoch 2, loss 0.9779769778251648
epoch 2, loss 0.9661227464675903
epoch 2, loss 0.9531617760658264
epoch 2, loss 0.9389583468437195
epoch 2, loss 0.9240256547927856
epoch 2, loss 0.910552442073822
epoch 2, loss 0.8976549506187439
epoch 2, loss 0.885875403881073
epoch 2, loss 0.8734304904937744
epoch 3, loss 0.8

In [242]:
yhat[0:10]

tensor([[-1.8671,  3.1897, -2.3804],
        [-7.6598, -0.0838, 10.3663],
        [-2.5126,  2.9921, -1.1193],
        [ 4.9552, -3.4120, -5.1994],
        [-7.8109, -0.1741, 10.7035],
        [-1.8369,  3.1618, -2.4086],
        [ 4.9045, -3.3656, -5.1474],
        [ 4.8406, -3.3071, -5.0818],
        [ 4.4102, -2.9133, -4.6403],
        [-1.8369,  3.1618, -2.4086]], grad_fn=<SliceBackward>)

In [243]:
torch.max(yhat.data,1)

(tensor([ 3.1897, 10.3663,  2.9921,  4.9552, 10.7035,  3.1618,  4.9045,  4.8406,
          4.4102,  3.1618,  3.4701,  4.2751,  4.2784,  3.9809,  3.2964,  4.7708,
          3.1618,  7.7798,  4.4911,  3.1618,  7.6823,  5.7390,  2.1863,  6.3554,
          3.1618,  2.8826,  7.2600,  3.1858,  4.8063,  3.1618,  6.1057,  4.7368,
          4.8365,  3.1618,  2.4657,  4.5940,  8.4205,  4.7318,  4.0456,  2.3036,
          3.1618,  8.2335,  3.2886,  4.2244,  9.4911,  3.2984,  4.1440,  4.6490,
          4.2619,  4.6560,  4.3067,  4.7849,  4.3284,  2.2233,  8.9186,  4.5002,
          6.6282, 10.7451,  4.6248,  3.1157,  3.1026,  2.7165,  3.1618,  7.3017,
          5.0349,  5.4656,  3.2933,  4.8900,  3.1245,  3.2993,  3.3014,  3.5692,
          3.2099,  3.2770,  5.1179,  3.1618,  7.3258,  6.4010,  4.6993,  3.1618,
          9.9448,  2.8461,  4.5940,  9.5094,  4.4287,  3.3045,  7.4998,  5.9506,
          3.1863,  4.5837,  3.1005,  3.1618,  6.2036,  3.8563,  4.6985,  3.1618,
          4.4315,  5.8660,  

In [244]:
y

tensor([1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

In [245]:
z = model(x_val)
z_dropout = model_drop(x_val)

In [246]:
_,yhat=torch.max(z.data,1)
yhat[0:20]

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2])

In [247]:
_,yhat_dropout=torch.max(z_dropout.data,1)
yhat_dropout[0:20]

tensor([2, 0, 2, 1, 2, 0, 1, 1, 2, 1, 2, 0, 1, 0, 0, 1, 1, 1, 2, 1])

In [248]:
y_val[0:20]

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2])

In [249]:
# Making the Confusion Matrix
eval_matrix = (pd.crosstab(y_val, yhat))
print(eval_matrix)

col_0   0   1   2
row_0            
0      19   0   0
1       0  15   0
2       0   1  15


In [250]:
# Making the Confusion Matrix
eval_matrix_dropout = (pd.crosstab(y_val, yhat_dropout))
print(eval_matrix_dropout)

col_0   0   1  2
row_0           
0      18   1  0
1       1  10  4
2       0   8  8


In [251]:
(eval_matrix[0][0]+eval_matrix[1][1]+eval_matrix[2][2])/y_val.shape[0]

0.97999999999999998

In [252]:
(eval_matrix_dropout[0][0]+eval_matrix_dropout[1][1]+eval_matrix_dropout[2][2])/y_val.shape[0]

0.71999999999999997